# Best NBA Defender - how they guard the greats

I want to know who the best NBA defenders are, as defined by who defends the great scorers the most effectively?

In [12]:
#Load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
# NBA API
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import teams

## Best Scorers in the NBA, 2022

In [ ]:
gen21 = pd.read_excel('player_general_outputs\\traditional_SeasonType_Regular_20Season_SeasonYear_2021-22.xlsx')


In [11]:
gen21 = gen21.sort_values('PTS', ascending=False)
gen21.head(10)

,Unnamed: 0,Unnamed: 1,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
1,1,1.0,Joel Embiid,PHI,28.0,68.0,45.0,23.0,33.8,30.6,...,11.7,4.2,3.1,1.1,1.5,2.7,55.5,46.0,2.0,5.4
2,2,2.0,LeBron James,LAL,37.0,56.0,25.0,31.0,37.2,30.3,...,8.2,6.2,3.5,1.3,1.1,2.2,53.0,21.0,6.0,-2.1
3,3,3.0,Giannis Antetokounmpo,MIL,27.0,67.0,45.0,22.0,32.9,29.9,...,11.6,5.8,3.3,1.1,1.4,3.2,56.5,46.0,4.0,5.9
4,4,4.0,Kevin Durant,BKN,33.0,55.0,36.0,19.0,37.2,29.9,...,7.4,6.4,3.5,0.9,0.9,2.1,50.3,18.0,4.0,4.9
5,5,5.0,Luka Doncic,DAL,23.0,65.0,44.0,21.0,35.4,28.4,...,9.1,8.7,4.5,1.2,0.6,2.2,53.1,44.0,10.0,2.2
6,6,6.0,Trae Young,ATL,23.0,76.0,40.0,36.0,34.9,28.4,...,3.7,9.7,4.0,0.9,0.1,1.7,46.5,42.0,0.0,2.1
7,7,7.0,DeMar DeRozan,CHI,32.0,76.0,43.0,33.0,36.1,27.9,...,5.2,4.9,2.4,0.9,0.3,2.3,42.7,6.0,0.0,1.0
8,8,8.0,Kyrie Irving,BKN,30.0,29.0,14.0,15.0,37.6,27.4,...,4.4,5.8,2.5,1.4,0.6,2.8,44.8,2.0,0.0,4.2
9,9,9.0,Ja Morant,MEM,22.0,57.0,36.0,21.0,33.1,27.4,...,5.7,6.7,3.4,1.2,0.4,1.5,45.6,10.0,1.0,3.3
10,10,10.0,Nikola Jokic,DEN,27.0,74.0,46.0,28.0,33.5,27.1,...,13.8,7.9,3.8,1.5,0.9,2.6,58.6,66.0,19.0,6.0


In [16]:
top_10 = gen21.loc[gen21['PTS'] > 27.0]
top_10_players = list(top_10['PLAYER'])

I am going to use these ten players as the offensive subjects to analyze defenders:
 - Joel Embiid (Center)
 - LeBron James (PF)
 - Giannis Antetokounmpo (PF)
 - Kevin Durant (PF)
 - Luka Doncic (SF)
 - Trae Young (PG)
 - DeMar Derozan (SF)
 - Kyrie Irving (PG)
 - Ja Morant (PG)
 - Nikola Jokic (Center)

In [13]:
stats = players.get_players()
stats = pd.DataFrame(stats)
stats.head()


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [17]:
# Add id to top_10

top_10.head()

,Unnamed: 0,Unnamed: 1,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
1,1,1.0,Joel Embiid,PHI,28.0,68.0,45.0,23.0,33.8,30.6,...,11.7,4.2,3.1,1.1,1.5,2.7,55.5,46.0,2.0,5.4
2,2,2.0,LeBron James,LAL,37.0,56.0,25.0,31.0,37.2,30.3,...,8.2,6.2,3.5,1.3,1.1,2.2,53.0,21.0,6.0,-2.1
3,3,3.0,Giannis Antetokounmpo,MIL,27.0,67.0,45.0,22.0,32.9,29.9,...,11.6,5.8,3.3,1.1,1.4,3.2,56.5,46.0,4.0,5.9
4,4,4.0,Kevin Durant,BKN,33.0,55.0,36.0,19.0,37.2,29.9,...,7.4,6.4,3.5,0.9,0.9,2.1,50.3,18.0,4.0,4.9
5,5,5.0,Luka Doncic,DAL,23.0,65.0,44.0,21.0,35.4,28.4,...,9.1,8.7,4.5,1.2,0.6,2.2,53.1,44.0,10.0,2.2


In [18]:
merged = pd.merge(top_10, stats, how = 'left', left_on = 'PLAYER', right_on= 'full_name')

In [19]:
merged

,Unnamed: 0,Unnamed: 1,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,...,PF,FP,DD2,TD3,+/-,id,full_name,first_name,last_name,is_active
0,1,1.0,Joel Embiid,PHI,28.0,68.0,45.0,23.0,33.8,30.6,...,2.7,55.5,46.0,2.0,5.4,203954,Joel Embiid,Joel,Embiid,True
1,2,2.0,LeBron James,LAL,37.0,56.0,25.0,31.0,37.2,30.3,...,2.2,53.0,21.0,6.0,-2.1,2544,LeBron James,LeBron,James,True
2,3,3.0,Giannis Antetokounmpo,MIL,27.0,67.0,45.0,22.0,32.9,29.9,...,3.2,56.5,46.0,4.0,5.9,203507,Giannis Antetokounmpo,Giannis,Antetokounmpo,True
3,4,4.0,Kevin Durant,BKN,33.0,55.0,36.0,19.0,37.2,29.9,...,2.1,50.3,18.0,4.0,4.9,201142,Kevin Durant,Kevin,Durant,True
4,5,5.0,Luka Doncic,DAL,23.0,65.0,44.0,21.0,35.4,28.4,...,2.2,53.1,44.0,10.0,2.2,1629029,Luka Doncic,Luka,Doncic,True
5,6,6.0,Trae Young,ATL,23.0,76.0,40.0,36.0,34.9,28.4,...,1.7,46.5,42.0,0.0,2.1,1629027,Trae Young,Trae,Young,True
6,7,7.0,DeMar DeRozan,CHI,32.0,76.0,43.0,33.0,36.1,27.9,...,2.3,42.7,6.0,0.0,1.0,201942,DeMar DeRozan,DeMar,DeRozan,True
7,8,8.0,Kyrie Irving,BKN,30.0,29.0,14.0,15.0,37.6,27.4,...,2.8,44.8,2.0,0.0,4.2,202681,Kyrie Irving,Kyrie,Irving,True
8,9,9.0,Ja Morant,MEM,22.0,57.0,36.0,21.0,33.1,27.4,...,1.5,45.6,10.0,1.0,3.3,1629630,Ja Morant,Ja,Morant,True
9,10,10.0,Nikola Jokic,DEN,27.0,74.0,46.0,28.0,33.5,27.1,...,2.6,58.6,66.0,19.0,6.0,203999,Nikola Jokic,Nikola,Jokic,True


### Gather matchup data

NBA.com matchup data is available here:
https://www.nba.com/stats/player/1628369/head-to-head?Matchup=Offense

In [24]:
# First, lets add that to the merged dataframe

merged['matchup_url'] = 'https://www.nba.com/stats/player/' + merged['id'].astype(str) + '/head-to-head?Matchup=Offense'

In [25]:
merged

,Unnamed: 0,Unnamed: 1,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,...,FP,DD2,TD3,+/-,id,full_name,first_name,last_name,is_active,matchup_url
0,1,1.0,Joel Embiid,PHI,28.0,68.0,45.0,23.0,33.8,30.6,...,55.5,46.0,2.0,5.4,203954,Joel Embiid,Joel,Embiid,True,https://www.nba.com/stats/player/203954/head-t...
1,2,2.0,LeBron James,LAL,37.0,56.0,25.0,31.0,37.2,30.3,...,53.0,21.0,6.0,-2.1,2544,LeBron James,LeBron,James,True,https://www.nba.com/stats/player/2544/head-to-...
2,3,3.0,Giannis Antetokounmpo,MIL,27.0,67.0,45.0,22.0,32.9,29.9,...,56.5,46.0,4.0,5.9,203507,Giannis Antetokounmpo,Giannis,Antetokounmpo,True,https://www.nba.com/stats/player/203507/head-t...
3,4,4.0,Kevin Durant,BKN,33.0,55.0,36.0,19.0,37.2,29.9,...,50.3,18.0,4.0,4.9,201142,Kevin Durant,Kevin,Durant,True,https://www.nba.com/stats/player/201142/head-t...
4,5,5.0,Luka Doncic,DAL,23.0,65.0,44.0,21.0,35.4,28.4,...,53.1,44.0,10.0,2.2,1629029,Luka Doncic,Luka,Doncic,True,https://www.nba.com/stats/player/1629029/head-...
5,6,6.0,Trae Young,ATL,23.0,76.0,40.0,36.0,34.9,28.4,...,46.5,42.0,0.0,2.1,1629027,Trae Young,Trae,Young,True,https://www.nba.com/stats/player/1629027/head-...
6,7,7.0,DeMar DeRozan,CHI,32.0,76.0,43.0,33.0,36.1,27.9,...,42.7,6.0,0.0,1.0,201942,DeMar DeRozan,DeMar,DeRozan,True,https://www.nba.com/stats/player/201942/head-t...
7,8,8.0,Kyrie Irving,BKN,30.0,29.0,14.0,15.0,37.6,27.4,...,44.8,2.0,0.0,4.2,202681,Kyrie Irving,Kyrie,Irving,True,https://www.nba.com/stats/player/202681/head-t...
8,9,9.0,Ja Morant,MEM,22.0,57.0,36.0,21.0,33.1,27.4,...,45.6,10.0,1.0,3.3,1629630,Ja Morant,Ja,Morant,True,https://www.nba.com/stats/player/1629630/head-...
9,10,10.0,Nikola Jokic,DEN,27.0,74.0,46.0,28.0,33.5,27.1,...,58.6,66.0,19.0,6.0,203999,Nikola Jokic,Nikola,Jokic,True,https://www.nba.com/stats/player/203999/head-t...


In [26]:
#os.mkdir('top_defenders')

In [27]:
matchup_url = merged.matchup_url

AttributeError: 'NoneType' object has no attribute 'findAll'